In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

calgary_weather_data_one_path = "./datasets/calgary_weather_data_20180101_20220101.csv"
calgary_weather_data_two_path = "./datasets/calgary_weather_data_20220101_20250101.csv"

cw1_df = pd.read_csv(calgary_weather_data_one_path, encoding="latin-1")
cw2_df = pd.read_csv(calgary_weather_data_two_path, encoding="latin-1")

cw1_df = cw1_df[["Station Name", "Date (Local Standard Time)", "Air Temp. Avg. (°C)"]]
cw2_df = cw2_df[["Station Name", "Date (Local Standard Time)", "Air Temp. Avg. (°C)"]]

cw_df = pd.concat([cw1_df, cw2_df],  ignore_index=True)

cw_df[['Month', 'Year']] = cw_df['Date (Local Standard Time)'].str.split('-', expand=True)

cw_df['Month'] = pd.to_datetime(cw_df['Month'], format='%B').dt.month

cw_df['Year'] = cw_df['Year'].astype(int)

cw_df['Avg Temp'] = cw_df['Air Temp. Avg. (°C)'].astype(float)

cw_df = cw_df.drop(columns=['Date (Local Standard Time)', 'Air Temp. Avg. (°C)'])

display(cw_df.head())

In [ ]:
community_crime_statistics_calgary_path = "./datasets/community_crime_statistics_calgary.csv"

ccsc_df = pd.read_csv(community_crime_statistics_calgary_path)



In [ ]:
communities_by_ward_calgary_path = "./datasets/communities_by_ward_calgary.csv"
coms_ward_df = pd.read_csv(communities_by_ward_calgary_path)

coms_ward_df = coms_ward_df[["NAME", "WARD_NUM"]]

coms_ward_df.rename(columns={"NAME":"Community"}, inplace = True)

display(coms_ward_df)

In [ ]:
## For Prince
# Vancouver Crime Dataset

# Load dataset file paths
file_paths = [
    "./datasets/crimedata_csv_AllNeighbourhoods_2019.csv",
    "./datasets/crimedata_csv_AllNeighbourhoods_2020.csv",
    "./datasets/crimedata_csv_AllNeighbourhoods_2021.csv",
    "./datasets/crimedata_csv_AllNeighbourhoods_2022.csv",
    "./datasets/crimedata_csv_AllNeighbourhoods_2023.csv",
    "./datasets/crimedata_csv_AllNeighbourhoods_2024.csv",
]
# Load all datasets into a list
data = [pd.read_csv(file) for file in file_paths]

# Concatenate all datasets into single dataframe
joined_data = pd.concat(data, ignore_index=True)

# Save combined dataframe into a new CSV file
joined_data.to_csv("joined_crime_data.csv", index=False)


# Display Dataframe
display(joined_data.shape)

# Display Data head and tail
display(joined_data.head())
display(joined_data.tail())

print(" ")
# Group data by YEAR and TYPE to get crime counts
crime_counts = joined_data.groupby(['YEAR', 'TYPE']).size().unstack(fill_value=0)


# Plot bar chart
fig, ax = plt.subplots(figsize=(14, 7))
crime_counts.plot(kind='bar', stacked=False, width=0.8, ax=ax)

# Labels and title
plt.xlabel("Year")
plt.ylabel("Total Crime Count")
plt.title(" VANCOUVER Total Crime Counts per Year by Crime Type")
plt.legend(title="Crime Category", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.show()

In [ ]:
## For Ravin
community_disorder_statistics_calgary_path = "./datasets/community_disorder_statistics_calgary.csv"
cds_calgary_df = pd.read_csv(community_disorder_statistics_calgary_path)

##merge community disorder dataset with coms_ward datset on community then count all values
cd_data_with_coms_ward = cds_calgary_df.merge(coms_ward_df[['Community', 'WARD_NUM']], on='Community', how='left')
print(cd_data_with_coms_ward)




#Merge ward number into community crime dataframe
community_crime_with_ward_df = ccsc_df.merge(coms_ward_df[['Community','WARD_NUM']], on="Community", how="left")

display(community_crime_with_ward_df)

#Merge weather data into community crime data frame
community_crime_with_ward_weather_df = community_crime_with_ward_df.merge(cw_df[['Year','Month','Avg Temp']], on=['Year', 'Month'], how="left")

display(community_crime_with_ward_weather_df)





####
def crime_frequency_by_year(df):

    #sort dataframe by years
    sorted_years = sorted(df['Year'].unique()) 
    

    #loop through each year 
    for year in sorted_years:

        #select the year 
        crime_year = df[df['Year'] == year]
        
        #group the dataframe by category and month and sum all values in crimecount for that category and month
        crime_grouped = crime_year.groupby(['Category', 'Month'])['Crime Count'].sum().reset_index()

        #Create a new figure for the plot
        plt.figure(figsize=(12, 6))

        #loop through each unique category, and create a line for each category
        for category in crime_grouped['Category'].unique():
            subset = crime_grouped[crime_grouped['Category'] == category]
            plt.plot(subset['Month'], subset['Crime Count'], marker='o', label=category)

        #establish the labels
        plt.xlabel("Month")
        plt.ylabel("Crime Count")
        plt.title(f"Crime Frequency Distribution by Category ({year})")
        plt.xticks(range(1, 13))  
        plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1))  
        plt.show()


#call function with community crime ward weather dataframe 
crime_frequency_by_year(community_crime_with_ward_weather_df)


def crime_by_ward_by_year_heatmap(df):

    #sort the years in the dataframe
    sorted_years = sorted(df['Year'].unique()) 

    #loop through each year 
    for year in sorted_years:

        #select the year 
        crime_year_df = df[df['Year'] == year]

        #group the data by the ward number and category, then sum up the crime counts for each ward and category 
        crime_by_ward_category_year = crime_year_df.groupby(['WARD_NUM', 'Category'])['Crime Count'].sum().unstack()

        #create a heatmap using seaborn
        plt.figure(figsize=(12, 6))
        sns.heatmap(crime_by_ward_category_year, cmap="YlGnBu", linewidths=0.5, annot=True, fmt=".0f")

        #establish labels 
        plt.xlabel("Crime Category")
        plt.ylabel("Ward Number")
        plt.title(f"Crime Frequency by Ward and Crime Category ({year})")
        plt.show

#call function with community crime with ward weather dataset 
crime_by_ward_by_year_heatmap(community_crime_with_ward_df)



def crime_count_by_ward_aggregated_by_year_heatmap(df):
   
 
    #group by ward number and year, then sum up all crime counts 
    crime_count_by_ward_aggregated_by_year = (df.groupby(['WARD_NUM', 'Year'])['Crime Count'].sum().reset_index())

    #pivot the data
    crime_count_by_ward_aggregated_heatmap_data = crime_count_by_ward_aggregated_by_year.pivot(index='WARD_NUM', columns='Year', values='Crime Count')

    #create the heatmap using seaborn
    plt.figure(figsize=(12, 6))
    sns.heatmap(crime_count_by_ward_aggregated_heatmap_data, cmap="Blues", linewidths=0.5, annot=True, fmt=".0f")

    #create the labels 
    plt.xlabel("Year")
    plt.ylabel("Ward Number")
    plt.title("Crime Count Frequency by Ward and Year")
    plt.show()

#call the function with the community crime with ward weather and disorder dataset 
crime_count_by_ward_aggregated_by_year_heatmap(community_crime_with_ward_weather_df)



def plot_ward_event_heatmap_yearly(df):
    
    #group data by ward number and year and sum up all event counts for that ward number and year 
    heatmap_data = (df.groupby(['WARD_NUM', 'Year'])['Event Count'].sum().reset_index())

    #pivot data for the heatmap
    heatmap_data = df.pivot_table(index='WARD_NUM', columns='Year', values='Event Count', aggfunc='sum', fill_value=0)

    #set up plot
    plt.figure(figsize=(12, 8))
    sns.heatmap(heatmap_data, cmap="YlGnBu", annot=True, fmt=".0f", linewidths=0.5)

    #label data 
    plt.xlabel("Year")
    plt.ylabel("Ward Number")
    plt.title("Heatmap of Social Disorder Event Counts by Ward and Year")
    plt.show()


#call the function with the cd_data and wards dataset
plot_ward_event_heatmap_yearly(cd_data_with_coms_ward)



def plot_crime_counts_by_year(df):

    #group by category and year, sum up all crime counts for the category and year 
    crime_summary = df.groupby(['Category', 'Year'], as_index=False)['Crime Count'].sum()

    #set plot size
    plt.figure(figsize=(14, 8))
    sns.barplot(data=crime_summary, x='Year', y='Crime Count', hue='Category')

    #add labels 
    plt.xlabel("Year")
    plt.ylabel("Total Crime Count")
    plt.title("Total Crime Counts per Year by Crime Type")
    plt.legend(title="Crime Category", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


plot_crime_counts_by_year(community_crime_with_ward_df)

import geopandas as gpd


#load calgary boundary file
gdf = gpd.read_file("./datasets/Ward Boundaries_20250210.geojson")  # Update file path


gdf.rename(columns={'ward_num':'WARD_NUM'}, inplace=True)
display(gdf.head())

display(community_crime_with_ward_df.head())


print(gdf['WARD_NUM'].dtypes)
print(community_crime_with_ward_df['WARD_NUM'].dtypes)


gdf["WARD_NUM"] = gdf["WARD_NUM"].astype(float)

crime_summary_total = community_crime_with_ward_df.groupby("WARD_NUM")["Crime Count"].sum().reset_index()

#merge community crime with ward data, with the ward geojson data
gdf = gdf.merge(crime_summary_total, left_on="WARD_NUM", right_on="WARD_NUM")
print(gdf.head())

#calculate the center of each ward to blace labels 
gdf["centroid"] = gdf.geometry.centroid

#plot the map
fig, ax = plt.subplots(figsize=(12, 10))
gdf.plot(column="Crime Count", cmap="Reds", linewidth=0.8, edgecolor="black", legend=True, ax=ax)

#add labels for the ward number in the map
for idx, row in gdf.iterrows():
    centroid = row["centroid"]
    ax.text(centroid.x, centroid.y, str(int(row["WARD_NUM"])), fontsize=12, ha='center', color='black')

#Add a title for the data
ax.set_title("Crime Count by Ward in Calgary")

plt.show()